In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('titanic_train.csv')

In [3]:
test = pd.read_csv('titanic_test.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


##### Identifying the columns which are to be removes and nothing to do with prediction of the final result
1. Make PassengerId column as the Index in which submission file has the index column named as the PassengerId
2. Cabin : Since the Cabin column as  more than 20% of the missing values and we have to drop the column
3. Ticket : Ticket column is of alpha numeric value hence drop the Ticket column
4. Name : Since the name is of string type and It is a independent quantity hence remove the column Name

In [5]:
train.set_index('PassengerId', inplace = True)

In [6]:
train.drop(columns = ['Cabin','Ticket','Name'], inplace = True)

In [7]:
test.drop(columns = ['Cabin','Ticket','Name'], inplace = True)

In [8]:
feature_data =  train.drop(columns = ['Survived'])

In [9]:
target_data = train.Survived

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

In [11]:
int_data = feature_data.select_dtypes(include = ['int64'])
float_data = feature_data.select_dtypes(include = ['float'])
cat_data = feature_data.select_dtypes(include= ["O"])

In [12]:
float_pipeline = make_pipeline(SimpleImputer(strategy = 'mean'),MinMaxScaler())
int_pipeline = make_pipeline(SimpleImputer(strategy = 'mean'),MinMaxScaler())
cat_pipeline =  make_pipeline(SimpleImputer(strategy = 'most_frequent'), OrdinalEncoder())

In [13]:
from sklearn.compose import make_column_transformer

In [14]:
preprocessor = make_column_transformer(
(int_pipeline, int_data.columns),
    (float_pipeline, float_data.columns),
    (cat_pipeline, cat_data.columns)
    
)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
pipeline = make_pipeline(preprocessor, DecisionTreeClassifier())

In [19]:
pipeline.fit(trainX, trainY)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                             

In [20]:
pipeline.score(testX, testY)

0.7623318385650224

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [22]:
len(models
   )

NameError: name 'models' is not defined

In [23]:
models = [DecisionTreeClassifier(),RandomForestClassifier(n_estimators=1000),LinearRegression(),LogisticRegression(),GaussianNB()]
names = ['decisiontree','randomforest', 'linearregression','logisticregression','gausian']

for name , estimator in zip(names, models):
    pipeline = make_pipeline(preprocessor, estimator)
    pipeline.fit(trainX, trainY)
    print(pipeline.score(testX, testY), name)
    

0.7399103139013453 decisiontree
0.8026905829596412 randomforest
0.32201060054954467 linearregression
0.8026905829596412 logisticregression
0.7847533632286996 gausian


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### From the above observation we found the Random forest has given more accuracy than the other algorithms

#### Since the Randomforest algoritthm given the highest accuracy hence we try to improve the accuracy by the feature_selection

### Feature Selection

In [24]:
from sklearn.feature_selection import SelectKBest,f_classif

In [25]:
popline = make_pipeline(preprocessor,MinMaxScaler(),SelectKBest(k =7, score_func = f_classif),RandomForestClassifier(n_estimators= 1000))

In [26]:
for i in range(1,8):
    popline = make_pipeline(preprocessor,MinMaxScaler(),SelectKBest(k =i, score_func = f_classif),RandomForestClassifier(n_estimators= 1000))
    popline.fit(trainX, trainY)
    print(i, popline.score(testX, testY))
    

1 0.7802690582959642
2 0.7713004484304933
3 0.7937219730941704
4 0.7847533632286996
5 0.7847533632286996
6 0.7802690582959642
7 0.8026905829596412


#### Conclusion , With 8 no. of features we are getting the highest score i.e 85% hence we opt the Randomforest as the final Alogirthm

In [ ]:
test.set_index('PassengerId', inplace = True)

In [ ]:
y_predit = pipeline.predict(test)

In [ ]:
submit_data = pd.DataFrame(y_predit, columns = ['Survived'])

In [ ]:
submit_data.set_index(test.index, inplace = True)

In [ ]:
submit_data =submit_data.to_csv()

In [ ]:
fp = open("text1.csv","w",encoding = "utf-8")
content1 = fp.write(submit_data)
fp.close()






In [ ]:
help(KNeighborsClassifier)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
pipeline = make_pipeline(preprocessor,KNeighborsClassifier(n_neighbors=  5))

In [ ]:
pipeline.fit(trainX, trainY)

In [ ]:
pipeline.score(testX, testY)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
pipeline  = make_pipeline(preprocessor,XGBClassifier())

In [ ]:
pipeline.fit(trainX, trainY)

In [ ]:
pipeline.score(testX, testY)

In [ ]:
submit_data2 =  pipeline.predict(test)

In [ ]:
submit_data2 = pd.DataFrame(submit_data2, columns = ['Survived'])


In [ ]:
test

In [ ]:
submit_data2.set_index(test.index, inplace = True)

In [ ]:
submit_data2 = submit_data2.to_csv()

In [ ]:
fp = open('submit_data_titanic.csv',"w+")
fp.write(submit_data2)
fp.close()

### Hyper parameter tuning using the gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.svm import SVC

In [ ]:
model_names = {
    'svm':{
        'model' : SVC(),
        'params' : {
            'C': [10,20],
            'kernal' :['rbf','linear']
            
        } 
           
    },
    'Random_forest':{
        'model' : RandomForestClassifier(),
        'params' :{
        'n_estimators' : [10, 25, 39]
        }
    },
    'Logistic_regression': {
        'model' : LogisticRegression(),
        
        'params' :{
            'C' : [10, 20]
        }
    }
}


In [ ]:
score = []
for name, mp in model_names.items():
    pp = GridSearchCV(mp['model'], mp['params'], cv = 5, return_train_score= False)
    pp.fit(trainX, trainY)
    score.apppend({
    'model': name,
    'best_score': pp.best_score,
    'best_params': pp.best_params_})
score

In [27]:
from sklearn.ensemble import AdaBoostClassifier

In [29]:
help(AdaBoostClassifier)

Help on class AdaBoostClassifier in module sklearn.ensemble.weight_boosting:

class AdaBoostClassifier(BaseWeightBoosting, sklearn.base.ClassifierMixin)
 |  AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)
 |  
 |  An AdaBoost classifier.
 |  
 |  An AdaBoost [1] classifier is a meta-estimator that begins by fitting a
 |  classifier on the original dataset and then fits additional copies of the
 |  classifier on the same dataset but where the weights of incorrectly
 |  classified instances are adjusted such that subsequent classifiers focus
 |  more on difficult cases.
 |  
 |  This class implements the algorithm known as AdaBoost-SAMME [2].
 |  
 |  Read more in the :ref:`User Guide <adaboost>`.
 |  
 |  Parameters
 |  ----------
 |  base_estimator : object, optional (default=None)
 |      The base estimator from which the boosted ensemble is built.
 |      Support for sample weighting is required, as well as proper
 |

In [31]:
pipline = make_pipeline(preprocessor, AdaBoostClassifier(base_estimator = RandomForestClassifier(), n_estimators = 100))

In [33]:
pipeline.fit(trainX, trainY)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                             

In [40]:
from sklearn.metrics import confusion_matrix, f1_score

In [39]:
confusion_matrix(y_pred=pipeline.predict(testX), y_true= testY)

array([[121,  22],
       [ 26,  54]], dtype=int64)

In [41]:
f1_score(pipeline.predict(testX), testY)

0.6923076923076923

In [34]:
pipeline.score(testX, testY)

0.7847533632286996